# YAML usage

<div class="alert alert-info"> <b>NOTE</b>

This tutorial will mostly remain xscen-specific and, thus, will not go into more advanced YAML functionalities such as anchors. More information on that can be consulted [here](https://support.atlassian.com/bitbucket-cloud/docs/yaml-anchors/), while [this template](https://github.com/Ouranosinc/xscen/blob/main/templates/1-basic_workflow_with_config/config1.yml) makes ample use of them.

</div>

While parameters can be explicitly given to functions, most support the use of YAML configuration files to automatically pass arguments. This tutorial will go over basic principles on how to write and prepare configuration files, and provide a few examples.

An `xscen` function supports YAML parametrisation if it is preceded by the `parse_config` wrapper in the code. Currently supported functions are:

In [ ]:
from xscen.config import get_configurable

list(get_configurable().keys())

## Loading an existing YAML config file

YAML files are read using `xscen.load_config`. Any number of files can be called, which will be merged together into a single python dictionary accessed through `xscen.CONFIG`.

In [ ]:
from pathlib import Path

import xscen as xs
from xscen import CONFIG

In [ ]:
# Load configuration
xs.load_config(
    str(
        Path().absolute().parent.parent
        / "templates"
        / "1-basic_workflow_with_config"
        / "config1.yml"
    ),
    # str(Path().absolute().parent.parent / "templates" / "1-basic_workflow_with_config" / "paths1_example.yml")  We can't actually load this file due to the fake paths, but this would be the format
)

# Display the dictionary keys
print(CONFIG.keys())

`xscen.CONFIG` behaves similarly to a python dictionary, but has a custom `__getitem__` that returns a `deepcopy` of the requested item. As such, it is unmutable and thus, reliable and robust.

In [ ]:
# A normal python dictionary is mutable, but a CONFIG dictionary is not.
pydict = dict(CONFIG["project"])
print(CONFIG["project"]["id"], ", ", pydict["id"])
pydict2 = pydict
pydict2["id"] = "modified id"
print(CONFIG["project"]["id"], ", ", pydict["id"], ", ", pydict2["id"])
pydict3 = pydict2
pydict3["id"] = "even more modified id"
print(
    CONFIG["project"]["id"],
    ", ",
    pydict["id"],
    ", ",
    pydict2["id"],
    ", ",
    pydict3["id"],
)

If one really want to modify the `CONFIG` dictionary from within the workflow itself, its `set` method must be used.

In [ ]:
CONFIG.set("project.id", "modified id")
print(CONFIG["project"]["id"])

## Building a YAML config file
### Generic arguments

Since `CONFIG` is a python dictionary, anything can be written in it if it is deemed useful for the execution of the script. A good practice, such as seen in [this template's config1.yml](https://github.com/Ouranosinc/xscen/tree/main/templates/1-basic_workflow_with_config/config1.yml), is for example to use the YAML file to provide a list of tasks to be accomplished, give the general description of the project, or provide a dask configuration:

In [ ]:
print(CONFIG["tasks"])
print(CONFIG["project"])
print(CONFIG["regrid"]["dask"])

These are not linked to any function and will not automatically be called upon by `xscen`, but can be referred to during the execution of the script. Below is an example where `tasks` is used to instruct on which tasks to accomplish and which to skip. Many such example can be seen throughout [the provided templates](https://github.com/Ouranosinc/xscen/tree/main/templates).

In [ ]:
if "extract" in CONFIG["tasks"]:
    print("This will start the extraction process.")

if "figures" in CONFIG["tasks"]:
    print(
        "This would start creating figures, but it will be skipped since it is not in the list of tasks."
    )

### Function-specific parameters

In addition to generic arguments, a major convenience of YAML files is that parameters can be automatically fed to functions if they are wrapped by `@parse_config` (see above for the list of currently supported functions). The exact following format has to be used:

```
module:
    function:
        argument:
```

The most up-to-date list of modules can be consulted [here](https://xscen.readthedocs.io/en/latest/apidoc/modules.html), as well as at the start of this tutorial. A simple example would be as follows:
```
aggregate:
  compute_deltas:
    kind: "+"
    reference_horizon: "1991-2020"
    to_level: 'delta'
```

Some functions have arguments in the form of lists and dictionaries. These are also supported:
```
extract:
    search_data_catalogs:
      variables_and_freqs:
        tasmax: D
        tasmin: D
        pr: D
        dtr: D
      allow_resampling: False
      allow_conversion: True
      periods: ['1991', '2020']
      other_search_criteria:
        source:
          "ERA5-Land"
```

In [ ]:
# Note that the YAML used here is more complex and separates tasks between 'reconstruction' and 'simulation', which would break the automatic passing of arguments.
print(
    CONFIG["extract"]["reconstruction"]["search_data_catalogs"]["variables_and_freqs"]
)  # Dictionary
print(CONFIG["extract"]["reconstruction"]["search_data_catalogs"]["periods"])  # List

Let's test that it is working, using `climatological_op`:

In [ ]:
# We should obtain 30-year means separated in 10-year intervals.
CONFIG["aggregate"]["climatological_op"]

In [ ]:
import pandas as pd
import xarray as xr

# Create a dummy dataset
time = pd.date_range("1951-01-01", "2100-01-01", freq="YS-JAN")
da = xr.DataArray([0] * len(time), coords={"time": time}, name="test")
ds = da.to_dataset()

# Call climatological_op using no argument other than what's in CONFIG
print(xs.climatological_op(ds))

### Managing paths

As a final note, it should be said that YAML files are a good way to privately provide paths to a script without having to explicitly write them in the code. [An example is provided here](https://github.com/Ouranosinc/xscen/blob/main/templates/1-basic_workflow_with_config/paths1_example.yml). As stated earlier, `xs.load_config` will merge together the provided YAML files into a single dictionary, meaning that the separation will be seamless once the script is running.

As an added protection, if the script is to be hosted on Github, `paths.yml` (or whatever it is being called) can then be added to the `.gitignore`.

### Configuration of external packages
As explained in the `load_config` [documentation](https://xscen.readthedocs.io/en/latest/api.html#special-sections), a few top-level sections can be used to configure packages external to xscen. For example, everything under the `logging` section will be sent to `logging.config.dictConfig(...)`, allowing the [full configuration](https://docs.python.org/3/library/logging.config.html#configuration-dictionary-schema) of python's built-in logging mechanism. The current config does exactly that by configuring a logger for `xscen` that logs to the console, with a sensibility set to the INFO level and a specified record formatting :

In [ ]:
CONFIG["logging"]

## Passing configuration through the command line
In order to have a more flexible configuration, it can be interesting to modify it using the command line. This way, the workflow can be started with different values without having to edit and save the YAML file each time. Alternatively, the command line arguments can also be used to determine which configuration file to use, so that the same workflow can be launched with different configurations without needing to duplicate the code. The [second template workflow](https://github.com/Ouranosinc/xscen/blob/main/templates/2-indicators_only/workflow2.py) uses this method.

The idea is simply to create an `ArgumentParser` with python's built-in [argparse](https://docs.python.org/3/library/argparse.html) :

In [ ]:
from argparse import ArgumentParser

parser = ArgumentParser(description="An example CLI arguments parser.")
parser.add_argument("-c", "--conf", action="append")

# Let's simulate command line arguments
example_args = (
    "-c ../../templates/2-indicators_only/config2.yml "
    '-c project.title="Title" '
    "--conf project.id=newID"
)

args = parser.parse_args(example_args.split())
print(args.conf)

And then we can simply pass this list to `load_config`, which accepts file paths and "key=value" pairs.

In [ ]:
xs.load_config(*args.conf)

print(CONFIG["project"]["title"])
print(CONFIG["project"]["id"])